In [33]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import pandas as pd
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
token_dict = pd.DataFrame(tk_dbrt.vocab.items(), columns=["token", "id"]).sort_values("id").reset_index(drop=True).set_index("id")

texto = "No Hola a todos"
tokens = tokenizer.tokenize(texto)
ids = tokenizer.convert_tokens_to_ids(tokens)

token_dict.loc[ids]

,token
id,
2053,no
7570,ho
2721,##la
1037,a
28681,tod
2891,##os


In [5]:
from src.datasets.text_datasets.old.tripadvisor.W2Vdataset import W2Vdataset
from src.models.text_models.W2V import W2V

base_path = "/media/nas/datasets/tripadvisor/restaurants/"
cities = ["gijon", "barcelona", "madrid"]

remove_numbers = True
remove_accents = True
remove_plurals = False
lemmatization = True
stemming = False

w2v_dimen = 128
seed = 100
gpu = 1

w2v_dts = W2Vdataset({"cities": cities, "city": "multi", "seed": seed, "data_path": base_path, "save_path": "data/",  # base_path + "Datasets/",
                        "remove_plurals": remove_plurals, "stemming": stemming, "lemmatization": lemmatization,
                        "remove_accents": remove_accents, "remove_numbers": remove_numbers,
                        }, load=[])
w2v_dts.CONFIG["dataset"]="multi"
w2v_dts.CONFIG["subset"]="-".join(cities)

In [8]:
w2v_dts.DATA

{}

In [7]:
w2v_mdl = W2V({"model": {"train_set": "ALL_TEXTS", "min_count": 100, "window": 5, "n_dimensions": w2v_dimen, "seed": seed},
                "session": {"gpu": gpu, "mixed_precision": True, "in_md5": False}}, w2v_dts)

w2v_mdl.train()

[WARNING] Model folder already exists...


KeyError: 'ALL_TEXTS'

In [4]:
# Imprimir los tokens
for token, idt in tk_dbrt.vocab.items():
    print(idt,token)

26771 accountable
4562 bear
29525 vet
11591 sheer
11861 ming
6314 upset
1225 ի
1717 ノ
7248 platforms
18953 scaled
28410 jalan
12969 mobility
23761 applicant
7476 ##ical
1905 瀬
18639 ##ulin
28368 ousted
22376 246
5914 marry
5321 dynasty
2503 inside
26267 282
28044 reunification
30027 ##ᴮ
29066 ghz
22946 tunisian
9350 rat
26443 ozone
5849 commissioner
30206 ##め
15435 ipswich
4549 netherlands
18808 ##tadt
3868 racing
3961 remain
856 [unused851]
17971 duffy
16872 ##vd
17028 ##bius
21388 bei
23183 qur
24336 ##enne
23594 barron
9312 evaluation
13477 ibrahim
21521 natalia
2705 ##th
29803 ##נ
16187 captivity
29223 appropriated
12935 cf
26171 qb
20936 ##omi
7579 belly
5895 prisoners
10369 pig
922 [unused917]
5483 greece
14342 unfinished
10620 damp
7109 horn
22835 ##imating
23019 truths
1143 ʷ
29973 ##မ
30237 ##ッ
28489 337
17751 ##mobile
30508 ##龍
7450 amendment
4801 ##ky
3018 awarded
3884 marine
11049 rebounds
28211 howie
11908 summoned
498 [unused493]
7597 prices
26832 ##lved
27459 aeronautics

In [1]:
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow as tf



2023-06-01 19:30:11.023814: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")
model = TFAutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish")


In [24]:
text = "Hola mundo"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
output[1]

(<tf.Tensor: shape=(2, 1, 12, 2, 64), dtype=float32, numpy=
 array([[[[[-5.38292527e-01,  2.48257160e-01, -5.22796094e-01, ...,
            -1.59949288e-01,  1.91841722e-02, -5.79061687e-01],
           [ 1.38699219e-01,  3.85100216e-01, -7.28040785e-02, ...,
             4.47283864e-01,  5.61189465e-02, -3.39824677e-01]],
 
          [[ 1.38657653e+00,  2.03933507e-01,  7.05375969e-01, ...,
             4.38653916e-01, -5.71687579e-01,  3.57128829e-01],
           [ 1.21856704e-02,  1.00735441e-01, -2.47440279e-01, ...,
            -7.77103603e-01,  7.35555053e-01,  2.79312193e-01]],
 
          [[-7.03806758e-01,  1.09546900e+00, -1.15335071e+00, ...,
            -4.36316460e-01, -7.29311705e-01, -3.45411867e-01],
           [ 2.81185806e-02,  4.98431563e-01, -1.37484014e-01, ...,
             4.44768310e-01,  9.50722992e-02,  9.51809168e-01]],
 
          ...,
 
          [[-1.44997388e-01,  1.06378722e+00, -4.78014983e-02, ...,
            -2.81589366e-02,  2.32391089e-01,  2.01418

In [13]:
encoded_input

{'input_ids': <tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[3041, 5372]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[1, 1]], dtype=int32)>}

In [11]:
preprocessor = hub.load( "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

# Step 1: tokenize batches of text inputs.
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string) # This SavedModel accepts up to 2 text inputs.
tokenize = hub.KerasLayer(preprocessor.tokenize)
tokenized_inputs = [tokenize(text_input)]

# Step 2: pack input sequences for the Transformer encoder.
seq_length = 200  # Your choice here.
bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs, arguments=dict(seq_length=seq_length))  # Optional argument.
encoder_inputs = bert_pack_inputs(tokenized_inputs)

# Model
encoder = hub.KerasLayer( "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2", trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 128].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 128].

In [23]:
embedding_model = tf.keras.Model(text_input, sequence_output)
sentences = tf.constant(["Hello world"])

res = embedding_model(sentences)